# Summarization models selection 🤖🤖🤖

## Libraries installation 📚

In [1]:
!pip install torch transformers
!pip install nltk
!pip install datasets
!pip install evaluate
!pip install numpy pandas scikit-learn
!pip install rouge_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1f6091987f963934f32c37f7a9b869529e4b61619aff27b40592f73f11d64516
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


## Libraries import 📚

In [2]:
import torch, nltk, transformers,evaluate, shutil, os
import numpy as np
import pandas as pd
from datasets import Dataset
from google.colab import files
from sklearn.model_selection import train_test_split
from transformers import pipeline,  DataCollatorForSeq2Seq, BartTokenizer, BartForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

## Functions definition 🧮

In [3]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["cleaned_dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [4]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred# Obtaining predictions and true labels

    # Decoding predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Obtaining the true labels tokens, while eliminating any possible masked token (i.e., label = -100)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Computing rouge score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value* 100 for key, value in result.items()} # Extracting some results

    # Add mean-generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

## Device setting 

In [12]:
if torch.cuda.is_available():
    print("GPU is available. \nUsing GPU")
    device = torch.device('cuda:0')
else:
    print("GPU is not available. \nUsing CPU")
    device = torch.device('cpu')

GPU is available. 
Using GPU


## File Upload and Dataframe manipulation 🗃️

In [5]:
# Upload the file
uploaded = files.upload()

# Read the CSV file
df = pd.read_csv('data.csv', sep=';')

Saving data.csv to data.csv


In [6]:
df = df.rename({'dialogue':'cleaned_dialogue'},axis=1)
df.head()

,summary,cleaned_dialogue
0,amanda baked cookies and will bring jerry some...,amanda i baked cookies do you want some jerry ...
1,olivia and olivier are voting for liberals in ...,olivia who are you voting for in this election...
2,kim may try the pomodoro technique recommended...,tim hi what is up kim bad relatable feeling to...
3,edward thinks he is in love with bella rachel ...,edward rachel i think i am in ove with bella r...
4,sam is confused because he overheard rick comp...,sam hey overheard rick say something sam i do ...


In [7]:
df.isnull().sum()

,0
summary,0
cleaned_dialogue,1


In [8]:
df = df.dropna()

#### Split dataframe in train and test sets

In [9]:
train_df, test_df = train_test_split(df, test_size=0.10, random_state=42)

#### Loading train and test Datasets

In [10]:
train_df['cleaned_dialogue'] = train_df['cleaned_dialogue'].astype(str)
train_df['summary'] = train_df['summary'].astype(str)
train_ds = Dataset.from_pandas(train_df)
train_ds

Dataset({
    features: ['summary', 'cleaned_dialogue', '__index_level_0__'],
    num_rows: 14731
})

In [11]:
test_df['cleaned_dialogue'] = test_df['cleaned_dialogue'].astype(str)
test_df['summary'] = test_df['summary'].astype(str)
test_ds = Dataset.from_pandas(test_df)
test_ds

Dataset({
    features: ['summary', 'cleaned_dialogue', '__index_level_0__'],
    num_rows: 1637
})

In [13]:
nltk.download('punkt', quiet=True)

metric = evaluate.load('rouge')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Metrics definition 📈

This part describes the metrics that were used to evaluate the large language models (LLMs) for the dialogue summarization task.

### 1. Training Loss
- **Description**: The average loss during model training. Lower values indicate that the model is learning well, minimizing the difference between predicted and actual values. Typically, this is computed over each batch during training and helps measure model convergence.

### 2. Validation Loss
- **Description**: The average loss during validation. This is a key metric to monitor overfitting. If validation loss diverges significantly from training loss, the model may not generalize well to unseen data. Lower values generally indicate better generalization.

### 3. ROUGE-1
- **Description**: Measures the overlap of unigrams (single words) between the generated summary and the reference summary. Useful for evaluating recall of individual words and content selection quality in summarization.

### 4. ROUGE-2
- **Description**: Measures the overlap of bigrams (two consecutive words) between the generated summary and the reference summary. Provides a more stringent measure of similarity and coherence, capturing phrase-level overlap.

### 5. ROUGE-L
- **Description**: Measures the longest common subsequence (LCS) between the generated and reference summaries. This metric captures the longest sequence of matching words while preserving order, making it useful for assessing fluency and coherence in the generated summary.

### 6. ROUGE-LSum
- **Description**: A variation of ROUGE-L tailored specifically for summarization tasks. It considers sentence-level longest common subsequence, measuring similarity between sentences in the generated and reference summaries. Useful for evaluating complex summarization structures.

### 7. Gen Len (Generation Length)
- **Description**: Captures the average length of the generated summaries. This metric helps understand whether the model is generating summaries that are appropriately concise or verbose relative to the desired output length.


## Model definition 🤖

### 1. BART (Bidirectional and Auto-Regressive Transformers)
- **Architecture**: BART is a transformer-based sequence-to-sequence model that combines a bidirectional encoder (similar to BERT) with an autoregressive decoder (similar to GPT). This design allows BART to capture context in both directions (left-to-right and right-to-left) during encoding, while generating text sequentially during decoding.
- **Strengths in Summarization**: BART excels in summarization due to its robust bidirectional encoding of context and autoregressive decoding, which together provide a strong foundation for generating coherent and contextually accurate summaries.

### 2. T5 (Text-to-Text Transfer Transformer)
- **Architecture**: T5 is a unified text-to-text framework that treats every NLP task as a text generation task. It is a transformer-based model with both encoder and decoder components. Unlike many models that specialize in specific tasks, T5 uses the same format (text-to-text) across all NLP tasks, allowing for straightforward task adaptation.
- **Strengths in Summarization**: T5’s text-to-text format allows it to handle summarization in a straightforward manner by simply transforming input text to summary text. Its flexibility makes it easy to adapt for different summarization styles or lengths, and it performs well in both abstractive and extractive summarization.

**BART Model**

In [24]:
checkpoint = 'facebook/bart-large-cnn' # Model
tokenizer = BartTokenizer.from_pretrained(checkpoint) # Loading Tokenizer
model = BartForConditionalGeneration.from_pretrained(checkpoint) # Loading Model
print(model)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [25]:
tokenized_train = train_ds.map(preprocess_function, batched=True,
                               remove_columns=['cleaned_dialogue', 'summary','__index_level_0__']) # Removing features

tokenized_test = test_ds.map(preprocess_function, batched=True,
                               remove_columns=['cleaned_dialogue', 'summary','__index_level_0__']) # Removing features

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1637 [00:00<?, ? examples/s]

In [26]:
# Instantiating Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Define arguments of the finetuning
seed = 42
training_args = Seq2SeqTrainingArguments(
    output_dir = './bart_samsum',
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    seed = seed,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.439700,1.438247,39.412700,19.779500,29.782800,29.786900,60.359800
2,0.869400,1.498526,39.653600,19.689100,29.979700,29.983700,60.288900
3,0.714300,1.586858,39.532400,19.241600,29.756300,29.760100,60.531500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=7364, training_loss=1.0528026997298408, metrics={'train_runtime': 6582.3598, 'train_samples_per_second': 8.952, 'train_steps_per_second': 1.119, 'total_flos': 2.600205747626803e+16, 'train_loss': 1.0528026997298408, 'epoch': 3.9989139288623403})

In [28]:
validation = trainer.evaluate(eval_dataset = tokenized_test)

In [29]:
# Define the directory name
directory = "bart_finetuned_samsum"

# Create the directory
! mkdir directory

# Save the model
trainer.save_model(directory)

# Save the model tokenizer
tokenizer.save_pretrained(directory)

# Saving the model in .zip format
shutil.make_archive(directory, 'zip', directory)

# Move the archive to the desired location if needed
shutil.move(f'{directory}.zip', '/content/bart_finetuned_samsum.zip')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


'/content/bart_finetuned_samsum.zip'

In [30]:
# Loading summarization pipeline and model
summarizer = pipeline('summarization', model = 'bart_finetuned_samsum',min_length=10, max_length=52)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [31]:
# Obtaining a random example from the validation dataset
test_ds[35]

{'summary': 'ryanair changed their bag policy adding a bag both ways is expensive lisa will show amy how to pack efficiently',
 'cleaned_dialogue': 'amy did you know that ryanair changed their bag policy lisa i know last time i flew i took only a little backpack with me lisa if you add a bag both ways it is not that cheap anymore amy and you managed to pack in a little backpack lisa i am a skilled traveller amy you need to teach me how to do it lisa travelling will teach you but i can give you a few tips',
 '__index_level_0__': 7498}

In [32]:
text = "amy did you know that ryanair changed their bag policy lisa i know last time i flew i took only a little backpack with me lisa if you add a bag both ways it is not that cheap anymore amy and you managed to pack in a little backpack lisa i am a skilled traveller amy you need to teach me how to do it lisa travelling will teach you but i can give you a few tips"
summary = "ryanair changed their bag policy adding a bag both ways is expensive lisa will show amy how to pack efficiently"
generated_summary = summarizer(text)

In [33]:
generated_summary

[{'summary_text': 'ryanair changed their bag policy lisa took a little backpack with her last time she flew lisa will teach amy how to do it'}]

**T5 Model**

In [14]:
checkpoint = 't5-base' # Model
tokenizer = T5Tokenizer.from_pretrained(checkpoint) # Loading Tokenizer
model = T5ForConditionalGeneration.from_pretrained(checkpoint) # Loading Model
print(model)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [15]:
tokenized_train = train_ds.map(preprocess_function, batched=True,
                               remove_columns=['cleaned_dialogue', 'summary','__index_level_0__']) # Removing features

tokenized_test = test_ds.map(preprocess_function, batched=True,
                               remove_columns=['cleaned_dialogue', 'summary','__index_level_0__']) # Removing features

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1637 [00:00<?, ? examples/s]

In [16]:
# Instantiating Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Define arguments of the finetuning
seed = 42
training_args = Seq2SeqTrainingArguments(
    output_dir = './t5',
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    seed = seed,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.593500,1.413125,42.735300,19.351800,35.505200,35.493700,17.296900
2,1.430300,1.352825,43.925200,20.506500,36.674600,36.682000,17.444100
3,1.408300,1.350868,43.871400,20.493400,36.664700,36.671000,17.463700


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=7364, training_loss=1.505748394453804, metrics={'train_runtime': 4276.9028, 'train_samples_per_second': 13.777, 'train_steps_per_second': 1.722, 'total_flos': 1.740469802171904e+16, 'train_loss': 1.505748394453804, 'epoch': 3.9989139288623403})

In [18]:
validation = trainer.evaluate(eval_dataset = tokenized_test)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [19]:
# Define the directory name
directory = "t5_finetuned_samsum"

# Create the directory
os.makedirs(directory, exist_ok=True)

# Save the model
trainer.save_model(directory)

# Save the model tokenizer
tokenizer.save_pretrained(directory)

# Saving the model in .zip format
shutil.make_archive(directory, 'zip', directory)

# Move the archive to the desired location if needed
shutil.move(f'{directory}.zip', '/content/t5_finetuned_samsum.zip')

'/content/t5_finetuned_samsum.zip'

In [20]:
# Loading summarization pipeline and model
summarizer = pipeline('summarization', model = 't5_finetuned_samsum',min_length=10, max_length=52)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [21]:
# Obtaining a random example from the validation dataset
test_ds[35]

{'summary': 'ryanair changed their bag policy adding a bag both ways is expensive lisa will show amy how to pack efficiently',
 'cleaned_dialogue': 'amy did you know that ryanair changed their bag policy lisa i know last time i flew i took only a little backpack with me lisa if you add a bag both ways it is not that cheap anymore amy and you managed to pack in a little backpack lisa i am a skilled traveller amy you need to teach me how to do it lisa travelling will teach you but i can give you a few tips',
 '__index_level_0__': 7498}

In [22]:
text = "amy did you know that ryanair changed their bag policy lisa i know last time i flew i took only a little backpack with me lisa if you add a bag both ways it is not that cheap anymore amy and you managed to pack in a little backpack lisa i am a skilled traveller amy you need to teach me how to do it lisa travelling will teach you but i can give you a few tips"
summary = "ryanair changed their bag policy adding a bag both ways is expensive lisa will show amy how to pack efficiently"
generated_summary = summarizer(text)

In [23]:
generated_summary

[{'summary_text': 'lisa took a little backpack with her last time she flew ryanair changed their bag policy'}]